In [55]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.metrics import mean_absolute_error
import os
import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import Ridge, Lasso, ElasticNet

import csv
import os
import json
import numpy as np
from datetime import datetime

In [56]:

folder = '../data/processed'
filename = 'dataset_final.csv'
file_path = os.path.join(folder, filename)

try:
    df = pd.read_csv(
        file_path,
        sep=',',
        decimal='.'
    )
    df = df.sort_values(by='DIA', ignore_index=True)
    print(f"✅ DataFrame cargado exitosamente desde: {file_path}")
    print(f"Dimensiones: {df.shape}")
    print("\nPrimeras 5 filas:")
    print(df.head())

except FileNotFoundError:
    print(f"❌ Error: No se encontró el archivo en la ruta: {file_path}")
except Exception as e:
    print(f"❌ Error al leer el archivo: {e}")

✅ DataFrame cargado exitosamente desde: ../data/processed\dataset_final.csv
Dimensiones: (1184, 394)

Primeras 5 filas:
          DIA  EE Planta / Hl  EE Elaboracion / Hl  EE Bodega / Hl  \
0  2020-07-01      642.727209            47.145349       69.023256   
1  2020-07-02        7.767254             0.769609        0.798838   
2  2020-07-03        8.801205             0.862593        0.835762   
3  2020-07-04        5.175639             0.439225        0.371077   
4  2020-07-05        7.924665             0.802365        0.717787   

   EE Cocina / Hl  EE Envasado / Hl  EE Linea 2 / Hl  EE Linea 3 / Hl  \
0        0.000000         13.813953        14.578784         0.000000   
1        0.319229          2.358593         4.158962         1.506838   
2        0.260924          1.985462        39.076667         1.448962   
3        0.258048          1.442114         4.348182         1.355238   
4        0.301592          1.664726         5.125920         2.704348   

   EE Linea 4 / Hl  

In [57]:

# 1️⃣ Cargar dataset definitivo
ruta_csv = "../data/processed/X_test_preproc.csv"
X_test = pd.read_csv(ruta_csv, sep=',', decimal='.')
ruta_csv = "../data/processed/X_train_preproc.csv"
X_train = pd.read_csv(ruta_csv, sep=',', decimal='.')
ruta_csv = "../data/processed/y_test.csv"
y_test = pd.read_csv(ruta_csv, sep=',', decimal='.')
ruta_csv = "../data/processed/y_train.csv"
y_train = pd.read_csv(ruta_csv, sep=',', decimal='.')


print("Shape dataset:", df.shape)




Shape dataset: (1184, 394)


In [58]:
def objective_rf_mae(trial):
    # Hiperparámetros
    n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=50)
    max_depth = trial.suggest_int("max_depth", 5, 50)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    max_features = trial.suggest_float("max_features", 0.1, 1.0, log=True)

    # Modelo
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        n_jobs=-1,
        random_state=42
    )
    
    # CV y Métrica (MAE)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="neg_mean_absolute_error", n_jobs=-1)
    
    return -np.mean(scores)

In [59]:
def objective_xgb_mae(trial):
    # Hiperparámetros
    n_estimators = trial.suggest_int("n_estimators", 100, 2000, step=100)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    subsample = trial.suggest_float("subsample", 0.6, 1.0, step=0.1)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.6, 1.0, step=0.1)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True) # L1
    reg_lambda = trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True) # L2

    # Modelo
    model = xgb.XGBRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        n_jobs=-1,
        random_state=42,
        verbosity=0
    )
    
    # CV y Métrica (MAE)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="neg_mean_absolute_error", n_jobs=-1)
    
    return -np.mean(scores)

In [60]:
def objective_lgbm_mae(trial):
    # Hiperparámetros
    n_estimators = trial.suggest_int("n_estimators", 100, 2000, step=100)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
    num_leaves = trial.suggest_int("num_leaves", 20, 150)
    subsample = trial.suggest_float("subsample", 0.6, 1.0, step=0.1)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.6, 1.0, step=0.1)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True) # L1
    reg_lambda = trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True) # L2

    # Modelo
    model = lgb.LGBMRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        num_leaves=num_leaves,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        n_jobs=-1,
        random_state=42,
        verbose=-1
    )
    
    # CV y Métrica (MAE)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="neg_mean_absolute_error", n_jobs=-1)
    
    return -np.mean(scores)

In [61]:
def objective_elasticnet_mae(trial):
    alpha = trial.suggest_float("alpha", 1e-3, 10.0, log=True)
    l1_ratio = trial.suggest_float("l1_ratio", 0.0, 1.0) 

    model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42, max_iter=2000)
    
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="neg_mean_absolute_error", n_jobs=-1)
    
    return -np.mean(scores)

# 1. Crear los estudios (todos buscan minimizar el MAE)
study_rf_mae = optuna.create_study(direction="minimize")
study_xgb_mae = optuna.create_study(direction="minimize")
study_lgbm_mae = optuna.create_study(direction="minimize")
study_elasticnet_mae = optuna.create_study(direction="minimize")

# 2. Ejecutar las optimizaciones
print("Optimizando Random Forest (MAE)...")
study_rf_mae.optimize(objective_rf_mae, n_trials=50)

print("Optimizando XGBoost (MAE)...")
study_xgb_mae.optimize(objective_xgb_mae, n_trials=75)

print("Optimizando LightGBM (MAE)...")
study_lgbm_mae.optimize(objective_lgbm_mae, n_trials=75)

print("Optimizando ElasticNet (MAE)...")
study_elasticnet_mae.optimize(objective_elasticnet_mae, n_trials=40)

# 3. Ver los mejores parámetros
print(f"Mejor RF (MAE): {study_rf_mae.best_value} con params {study_rf_mae.best_params}")
print(f"Mejor XGB (MAE): {study_xgb_mae.best_value} con params {study_xgb_mae.best_params}")
print(f"Mejor LGBM (MAE): {study_lgbm_mae.best_value} con params {study_lgbm_mae.best_params}")
print(f"Mejor ElasticNet (MAE): {study_elasticnet_mae.best_value} con params {study_elasticnet_mae.best_params}")

In [62]:

def log_experiment(model_name: str, 
                   model_object: object, 
                   study: optuna.study.Study, 
                   X_train, y_train, X_test, y_test):
    """
    Entrena un modelo con los mejores parámetros de un estudio de Optuna,
    calcula métricas y las guarda en un CSV.
    """
    
    # --- 1. Definir la ruta del log ---
    log_dir = "results"
    log_file = os.path.join(log_dir, "experiment_logs.csv")
    os.makedirs(log_dir, exist_ok=True) # Crea la carpeta si no existe

    # --- 2. Obtener mejores parámetros y valor del estudio ---
    best_params = study.best_params
    best_value = study.best_value # El MAE de CV que encontró Optuna
    
    # --- 3. Instanciar y entrenar el modelo final ---
    # Manejo especial para modelos lineales que necesitan Pipeline
    if "pipeline" in model_name.lower():
        # El pipeline ya está definido en la función objective, 
        # aquí solo re-creamos el mejor.
        if model_name == "Pipeline_ElasticNet":
             # Re-crea el pipeline con los mejores params
             model =ElasticNet(
                    alpha=best_params.get('alpha'), 
                    l1_ratio=best_params.get('l1_ratio'),
                    random_state=42, 
                    max_iter=2000
                )
            
            
    else:
        # Para modelos no-pipeline (RF, XGB, LGBM)
        model = model_object(**best_params, random_state=42, n_jobs=-1)
        # Ajustes para modelos que no les gusta 'n_jobs' o necesitan 'verbose'
        if isinstance(model, (xgb.XGBRegressor, lgb.LGBMRegressor)):
            model.set_params(verbosity=0) if isinstance(model, xgb.XGBRegressor) else None
            model.set_params(verbose=-1) if isinstance(model, lgb.LGBMRegressor) else None

    # Entrenar el modelo final con todos los datos de train
    model.fit(X_train, y_train)

    # --- 4. Calcular Métricas (Train y Test) ---
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    metrics = {
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'model_name': model_name,
        'optuna_cv_mae': best_value, # Métrica optimizada
        'mae_train': mean_absolute_error(y_train, y_pred_train),
        'mae_test': mean_absolute_error(y_test, y_pred_test),
        'rmse_train': np.sqrt(mean_squared_error(y_train, y_pred_train)),
        'rmse_test': np.sqrt(mean_squared_error(y_test, y_pred_test)),
        'r2_train': r2_score(y_train, y_pred_train),
        'r2_test': r2_score(y_test, y_pred_test),
        'hyperparameters': json.dumps(best_params) # Guardar params como string JSON
    }
    
    # --- 5. Escribir en el CSV ---
    file_exists = os.path.isfile(log_file)
    
    with open(log_file, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=metrics.keys())
        
        if not file_exists:
            writer.writeheader() # Escribir cabecera solo si el archivo es nuevo
            
        writer.writerow(metrics)
        
    print(f"Resultados de '{model_name}' guardados en {log_file}")

In [ ]:
# --- 1. Ejecutar el estudio (ejemplo con Random Forest) ---
print("Optimizando Random Forest (MAE)...")
study_rf_mae = optuna.create_study(direction="minimize")
# (Aquí asumimos que 'objective_rf_mae' está definida)
study_rf_mae.optimize(objective_rf_mae, n_trials=15) 

# --- 2. Registrar el resultado ---
log_experiment(
    model_name="RandomForest",
    model_object=RandomForestRegressor, # Pasa la clase del modelo
    study=study_rf_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)

# --- 5. Ejecutar el estudio (ejemplo con ElasticNet) ---
print("Optimizando ElasticNet (MAE)...")
study_elasticnet_mae = optuna.create_study(direction="minimize")
# (Aquí asumimos que 'objective_elasticnet_mae' está definida)
study_elasticnet_mae.optimize(objective_elasticnet_mae, n_trials=30)

# --- 6. Registrar el resultado ---
# Nota: 'model_object' es None porque se maneja dentro del 'if'
log_experiment(
    model_name="Pipeline_ElasticNet",
    model_object=None, 
    study=study_elasticnet_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)

# --- 3. Ejecutar el estudio (ejemplo con XGBoost) ---
print("Optimizando XGBoost (MAE)...")
study_xgb_mae = optuna.create_study(direction="minimize")
# (Aquí asumimos que 'objective_xgb_mae' está definida)
study_xgb_mae.optimize(objective_xgb_mae, n_trials=5)

# --- 4. Registrar el resultado ---
log_experiment(
    model_name="XGBoost",
    model_object=xgb.XGBRegressor, # Pasa la clase del modelo
    study=study_xgb_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)

# --- 5. Ejecutar el estudio (ejemplo con LGBM) ---
print("Optimizando LightGBM (MAE)...")
study_lgbm_mae = optuna.create_study(direction="minimize")
study_lgbm_mae.optimize(objective_lgbm_mae, n_trials=20)

# --- 6. Registrar el resultado ---
log_experiment(
    model_name="LightGBM",
    model_object=lgb.LGBMRegressor,  # Pasamos la clase del modelo
    study=study_lgbm_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)


[I 2025-11-15 20:07:41,158] A new study created in memory with name: no-name-99cc1422-81d4-40a3-953a-0f4c9122eb43


Optimizando Random Forest (MAE)...


[I 2025-11-15 20:07:46,233] Trial 0 finished with value: 8794.868952644205 and parameters: {'n_estimators': 800, 'max_depth': 26, 'min_samples_split': 20, 'min_samples_leaf': 5, 'max_features': 0.382744944042675}. Best is trial 0 with value: 8794.868952644205.
[I 2025-11-15 20:07:52,491] Trial 1 finished with value: 9130.866936223716 and parameters: {'n_estimators': 700, 'max_depth': 26, 'min_samples_split': 17, 'min_samples_leaf': 8, 'max_features': 0.5440833021062615}. Best is trial 0 with value: 8794.868952644205.
[I 2025-11-15 20:07:56,226] Trial 2 finished with value: 9048.109618885761 and parameters: {'n_estimators': 800, 'max_depth': 22, 'min_samples_split': 13, 'min_samples_leaf': 6, 'max_features': 0.12055965839796298}. Best is trial 0 with value: 8794.868952644205.
[I 2025-11-15 20:08:01,121] Trial 3 finished with value: 8957.270376592607 and parameters: {'n_estimators': 800, 'max_depth': 40, 'min_samples_split': 10, 'min_samples_leaf': 6, 'max_features': 0.36379599823290515}

Resultados de 'RandomForest' guardados en results\experiment_logs.csv
Optimizando ElasticNet (MAE)...


[I 2025-11-15 20:08:49,141] Trial 2 finished with value: 23072.861177262712 and parameters: {'alpha': 0.0128195557987465, 'l1_ratio': 0.9084916294320238}. Best is trial 1 with value: 11752.62750695349.
[I 2025-11-15 20:08:49,228] Trial 3 finished with value: 18685.90351140391 and parameters: {'alpha': 0.25766827467347786, 'l1_ratio': 0.9335806795812805}. Best is trial 1 with value: 11752.62750695349.
[I 2025-11-15 20:08:49,302] Trial 4 finished with value: 16522.88007302251 and parameters: {'alpha': 0.05414093544208757, 'l1_ratio': 0.2802993632803149}. Best is trial 1 with value: 11752.62750695349.
[I 2025-11-15 20:08:49,376] Trial 5 finished with value: 17788.222937618928 and parameters: {'alpha': 0.17755684693020293, 'l1_ratio': 0.8646818485078328}. Best is trial 1 with value: 11752.62750695349.
[I 2025-11-15 20:08:49,480] Trial 6 finished with value: 19917.47086297614 and parameters: {'alpha': 0.018073135386252413, 'l1_ratio': 0.4186048409971338}. Best is trial 1 with value: 11752.6

Resultados de 'Pipeline_ElasticNet' guardados en results\experiment_logs.csv
Optimizando XGBoost (MAE)...


[I 2025-11-15 20:08:59,466] Trial 0 finished with value: 10820.828515625 and parameters: {'n_estimators': 300, 'learning_rate': 0.015613882174437922, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.9, 'reg_alpha': 0.0930202174694126, 'reg_lambda': 3.063756646103939e-05}. Best is trial 0 with value: 10820.828515625.
[I 2025-11-15 20:09:38,282] Trial 1 finished with value: 10090.86435546875 and parameters: {'n_estimators': 1700, 'learning_rate': 0.04157739305572607, 'max_depth': 8, 'subsample': 1.0, 'colsample_bytree': 0.9, 'reg_alpha': 6.167985531906792e-06, 'reg_lambda': 0.2757408089166706}. Best is trial 1 with value: 10090.86435546875.
[I 2025-11-15 20:09:48,664] Trial 2 finished with value: 11567.62060546875 and parameters: {'n_estimators': 400, 'learning_rate': 0.1730432816958781, 'max_depth': 10, 'subsample': 0.8, 'colsample_bytree': 0.7, 'reg_alpha': 1.9843181063454414e-05, 'reg_lambda': 0.34472753791607885}. Best is trial 1 with value: 10090.86435546875.
[I 2025-11-15 20

Resultados de 'XGBoost' guardados en results\experiment_logs.csv


In [64]:
# --- 5. Ejecutar el estudio (ejemplo con LGBM) ---
print("Optimizando LightGBM (MAE)...")
study_lgbm_mae = optuna.create_study(direction="minimize")
study_lgbm_mae.optimize(objective_lgbm_mae, n_trials=10)

# --- 6. Registrar el resultado ---
log_experiment(
    model_name="LightGBM",
    model_object=lgb.LGBMRegressor,  # Pasamos la clase del modelo
    study=study_lgbm_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)

[I 2025-11-15 20:12:29,398] A new study created in memory with name: no-name-8225afee-40d9-4c2f-96fd-2d9186146f31


Optimizando LightGBM (MAE)...


[I 2025-11-15 20:12:49,102] Trial 0 finished with value: 19222.89854399785 and parameters: {'n_estimators': 1200, 'learning_rate': 0.041801902334070946, 'num_leaves': 108, 'subsample': 0.7, 'colsample_bytree': 0.9, 'reg_alpha': 3.1581322334369755e-05, 'reg_lambda': 1.3789582075354169e-05}. Best is trial 0 with value: 19222.89854399785.
[I 2025-11-15 20:13:21,397] Trial 1 finished with value: 15432.865046589644 and parameters: {'n_estimators': 1900, 'learning_rate': 0.013954417650184821, 'num_leaves': 149, 'subsample': 0.7, 'colsample_bytree': 0.8, 'reg_alpha': 0.00013457416109680852, 'reg_lambda': 0.0046723576597143554}. Best is trial 1 with value: 15432.865046589644.
[I 2025-11-15 20:13:39,009] Trial 2 finished with value: 24480.78065144061 and parameters: {'n_estimators': 1800, 'learning_rate': 0.138280995220693, 'num_leaves': 102, 'subsample': 0.6, 'colsample_bytree': 0.8, 'reg_alpha': 0.00011671153054042709, 'reg_lambda': 0.0001540583953702157}. Best is trial 1 with value: 15432.86

Resultados de 'LightGBM' guardados en results\experiment_logs.csv
